<a href="https://colab.research.google.com/github/MostakemHossain/pytorch-projects/blob/main/Text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers --quiet
!pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mostakemhossain
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection


In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
data_df=pd.read_json("/content/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json",lines=True)
print(data_df.shape)
data_df.dropna(inplace= True)
data_df.drop_duplicates(inplace = True)
data_df.drop(["article_link"],inplace=True,axis=1)
data_df.head()
print(data_df.shape)
data_df.head()

(26709, 3)
(26708, 2)


,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(data_df['headline']), np.array(data_df['is_sarcastic']), test_size=0.3)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

print("Training size",X_train.shape[0],"Rows is",round(X_train.shape[0]/data_df.shape[0],4)*100, "%")
print("Validation size",X_train.shape[0],"Rows is",round(X_val.shape[0]/data_df.shape[0],4)*100, "%")
print("Test size",X_train.shape[0],"Rows is",round(X_test.shape[0]/data_df.shape[0],4)*100, "%")

X_val.shape


Training size 18695 Rows is 70.0 %
Validation size 18695 Rows is 15.0 %
Test size 18695 Rows is 15.0 %


(4006,)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
bert_model = AutoModel.from_pretrained("google-bert/bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
class dataset(Dataset):
    def __init__(self, X, y):
        self.X = [tokenizer(x,
                           max_length=100,
                           truncation=True,
                           padding='max_length',
                           return_tensors='pt').to(device)
                  for x in X
                 ]
        self.y = torch.tensor(y, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = dataset(X_train, y_train)
test_dataset = dataset(X_test, y_test)
val_dataset = dataset(X_val, y_val)


In [ ]:
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-4


In [ ]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=BATCH_SIZE)
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=BATCH_SIZE)

In [ ]:
class my_model(nn.Module):
    def __init__(self, bert):
        super(my_model, self).__init__()

        self.bert = bert
        self.dropout = nn.Dropout(0.25)
        self.linear1 = nn.Linear(768, 384)
        self.linear2 = nn.Linear(384, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        pooled_output = self.bert(input_ids, attention_mask, return_dict=False)[0][:,0]
        output = self.linear1(pooled_output)
        output = self.dropout(output)
        output = self.linear2(output)
        output = self.dropout(output)
        output = self.sigmoid(output)

        return output


In [ ]:
for param in bert_model.parameters():
    param.requires_grad = False
model = my_model(bert_model).to(device)

In [ ]:
model

my_model(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [ ]:
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(),lr=LR)

In [ ]:
total_loss_train_plot = []
total_acc_train_plot = []
total_loss_val_plot = []
total_acc_val_plot = []

for epoch in range(EPOCHS):
    total_loss_train = 0
    total_acc_train = 0
    total_loss_val = 0
    total_acc_val = 0
    for data in train_dataloader:
        optimizer.zero_grad()
        inputs, labels = data

        preds = model(inputs['input_ids'].squeeze(1), inputs['attention_mask'].squeeze(1)).squeeze(1)

        loss = criterion(preds, labels)

        loss.backward()
        optimizer.step()

        total_loss_train += loss.item()
        total_acc_train += (preds.round() == labels).sum().item()

    with torch.no_grad():

        for data in val_dataloader:
            inputs, labels = data

            preds = model(inputs['input_ids'].squeeze(1), inputs['attention_mask'].squeeze(1)).squeeze(1)

            loss = criterion(preds, labels)

            total_loss_val += loss.item()
            total_acc_val += (preds.round() == labels).sum().item()


    total_loss_train_plot.append(round(total_loss_train/1000,4))
    total_acc_train_plot.append(round(total_acc_train/train_dataset.__len__() *100, 4))
    total_loss_val_plot.append(round(total_loss_val/1000,4))
    total_acc_val_plot.append(round(total_acc_val/val_dataset.__len__() *100, 4))

    print(f"epoch:{epoch}, train_loss:{round(total_loss_train/1000,4)}, train_acc{round(total_acc_train/train_dataset.__len__() *100, 4)}")
    print(f"epoch:{epoch}, val_loss:{round(total_loss_val/1000,4)}, val_acc{round(total_acc_val/val_dataset.__len__() *100, 4)}")

In [ ]:
with torch.no_grad():
    total_loss_test = 0
    total_acc_test = 0
    for data in test_dataloader:
            inputs, labels = data

            preds = model(inputs['input_ids'].squeeze(1), inputs['attention_mask'].squeeze(1)).squeeze(1)

            loss = criterion(preds, labels)

            total_loss_test += loss.item()
            total_acc_test += (preds.round() == labels).sum().item()

print(f"test_loss:{round(total_loss_test/1000,4)}, train_acc{round(total_acc_test/test_dataset.__len__() *100, 4)}")


In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(16,8))
axs[0].plot(total_loss_train_plot, label='training_loss')
axs[0].plot(total_loss_val_plot, label='validation_loss')
axs[0].legend()

axs[1].plot(total_acc_train_plot, label='training_acc')
axs[1].plot(total_acc_val_plot, label='val_acc')
axs[1].legend()